In [1]:
import pyspark
import math

In [2]:
sc = pyspark.SparkContext(appName="jojo")

In [3]:
sc

<SparkContext master=local[*] appName=jojo>

In [4]:
rdd = sc.sequenceFile("part-00000")

In [5]:
rdd.count()

36133

In [6]:
result = rdd.take(10)

In [8]:
def conta_palavras_doc(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_doc_freq = rdd.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)

In [38]:
def conta_uma_palavra(item, palavra):
    url, conteudo = item
    palavras = conteudo.strip().split()
    #count = 0
    lista = [palavra.lower() for palavra in palavras]
    if palavra in lista:
        return [item]
    return []


Rakin = rdd.filter(lambda x:"iphone" in x[1])
Yoda = rdd.filter(lambda x:"criança" in x[1])
inter = Rakin.intersection(Yoda)

rdd_Rakin = Rakin.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)
rdd_Yoda = Yoda.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)
rdd_inter = inter.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)

rdd_doc_rakin = Rakin.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)
rdd_doc_yoda = Yoda.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)
rdd_doc_inter = inter.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)

In [39]:
rdd_doc_rakin.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 31.0 failed 1 times, most recent failure: Lost task 9.0 in stage 31.0 (TID 311, JOSE-INSPER, executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:131)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:349)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:313)
	at org.apache.spark.api.python.PairwiseRDD.compute(PythonRDD.scala:103)
	at org.apache

In [ ]:
rdd_doc_yoda.count()

In [ ]:
rdd_doc_inter.count()

In [ ]:
rdd_doc_freq.count() #Numero de documentos

In [ ]:
N = rdd.count()

In [ ]:
DOC_COUNT_MIN = 5
DOC_COUNT_MAX = 0.7*N

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) and (contagem >= DOC_COUNT_MIN)

rdd_doc_rakin_freq_filtrado = rdd_doc_rakin.filter(filtra_doc_freq)

In [ ]:
rdd_doc_rakin_freq_filtrado.count() #Documentos filtrado no range definido

In [ ]:
def computa_idf(item):
    palavra,contagem = item
    idf = math.log10(N/contagem)
    return (palavra,idf)

rakin_rdd_idf = rdd_doc_rakin_freq_filtrado.map(computa_idf)

In [ ]:
result = rakin_rdd_idf.collect()

In [ ]:
sorted(result, key=lambda x: x[1])[:100]

In [ ]:
def conta_em_doc(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]